In [8]:
import csv
import cv2
import numpy as np

lines = []
with open('/Users/gongjianxin/Desktop/data_sample/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
            lines.append(line)

images = []
measurements = []
for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = '/Users/gongjianxin/Desktop/data_sample/IMG/' + filename 
    image = cv2.imread(current_path)
    images.append(image) 
    measurement= float(line[3])
    measurements.append(measurement)

augmented_images,augmented_measurements = [],[]
for image,measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
        
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout, Lambda
from keras.layers import Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x/255.0-0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(6, 5, 5, activation="relu"))
#model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(6, 5, 5, activation="relu"))
#model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2,shuffle = True,nb_epoch = 2)

model.save('model.h5')

Train on 12857 samples, validate on 3215 samples
Epoch 1/2
12857/12857 [==============================] - 518s - loss: 0.0611 - val_loss: 0.0136
Epoch 2/2
12857/12857 [==============================] - 490s - loss: 0.0110 - val_loss: 0.0112
